In [76]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import config 
import tmdbsimple as tmdb          # tmdbsimple 2.2.0
import time
tmdb.API_KEY = config.tmdb_api_key # get TMDB API key from config.py file

In [77]:
#get Dustin Hoffman's TMDB id
search_hoffman = tmdb.Search()      
response_hoffman = search_hoffman.person(query='Dustin Hoffman')
hoffman_id = search_hoffman.results[0]['id']
hoffman_id

4483

In [78]:
#Now get hoffman's credits
search_hoffman_id = tmdb.People(hoffman_id)
response_hoffman_id = search_hoffman_id.movie_credits()
response_hoffman_id


{'cast': [{'character': 'Raymond Babbitt',
   'credit_id': '52fe423cc3a36847f800e8b7',
   'poster_path': '/vllKYkA7eYi2UMM9yWQ1JegefBh.jpg',
   'id': 380,
   'video': False,
   'vote_count': 3317,
   'adult': False,
   'backdrop_path': '/eZ4Aj1O6E95Mk5O9oKKdvfnSWzf.jpg',
   'genre_ids': [18],
   'original_language': 'en',
   'original_title': 'Rain Man',
   'popularity': 17.248,
   'title': 'Rain Man',
   'vote_average': 7.7,
   'overview': "Selfish yuppie Charlie Babbitt's father left a fortune to his savant brother Raymond and a pittance to Charlie; they travel cross-country.",
   'release_date': '1988-12-11'},
  {'character': 'Stanley Motss',
   'credit_id': '52fe4258c3a36847f8016f15',
   'poster_path': '/cGatilylRyJiCoOu5T5sAB17HKR.jpg',
   'id': 586,
   'video': False,
   'vote_count': 459,
   'adult': False,
   'backdrop_path': '/bWoNwo04ZfZoEvEF8gliyB1EAyS.jpg',
   'genre_ids': [35, 18],
   'original_language': 'en',
   'original_title': 'Wag the Dog',
   'popularity': 10.386,
 

In [79]:
hoffman_n = len(response_hoffman_id['cast'][:])
hoffman_n   #how many movies

108

In [80]:
#Put all records in actor's first part of Data Frame (ie. hoffman_df)
labels = ['title','popularity','vote_average','vote_count']
idx = [film['id'] for film in response_hoffman_id['cast'][:]]
records = []
for film in (response_hoffman_id['cast'][:]):
    record = [film['title'], film['popularity'], film['vote_average'], film['vote_count']]
    records.append(record)

hoffman_df = pd.DataFrame.from_records(records, columns=labels, index=idx)
hoffman_df

,title,popularity,vote_average,vote_count
380,Rain Man,17.248,7.7,3317
586,Wag the Dog,10.386,6.9,459
693,Meet the Fockers,15.987,6.2,2795
819,Sleepers,13.808,7.5,1484
866,Finding Neverland,13.419,7.3,2352
879,Hook,17.253,6.9,3083
891,All the President's Men,21.243,7.7,807
994,Straw Dogs,12.695,7.2,406
1262,Stranger Than Fiction,8.599,7.2,1242
1427,Perfume: The Story of a Murderer,12.963,7.2,2355


In [81]:
#Now get 2nd part of actor's df from the tmdb.Movies for merging
labels = ['revenue','budget','profit','genres']
film_indices = [film[0] for film in hoffman_df.iterrows()]
#print(film_indices)
records = []
request_rate_limit = 40     #TMDB has rate limit of 40 requests per 10 seconds

for film_idx in film_indices:
    request_rate_limit -= 1
    if request_rate_limit == 0:            
        print("Hold yer horses, cowboy!")
        print("...wait per request limit...")
        time.sleep(10)
        request_rate_limit = 39
    movie = tmdb.Movies(film_idx)  #Each movie has unique identifier stored as index of df
    response = movie.info()
        
    profit = response['revenue'] - response['budget']
    record = [response['revenue'], response['budget'], profit, response['genres']]
    
    records.append(record)
#    if request_rate_limit <37: break

hoffman_movies_df = pd.DataFrame.from_records(records, columns=labels, index=film_indices)
hoffman_movies_df    


Hold yer horses, cowboy!
...wait per request limit...
Hold yer horses, cowboy!
...wait per request limit...


,revenue,budget,profit,genres
380,412800000,25000000,387800000,"[{'id': 18, 'name': 'Drama'}]"
586,64256513,15000000,49256513,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam..."
693,516642939,80000000,436642939,"[{'id': 35, 'name': 'Comedy'}, {'id': 10749, '..."
819,165615285,44000000,121615285,"[{'id': 80, 'name': 'Crime'}, {'id': 18, 'name..."
866,116766556,25000000,91766556,"[{'id': 18, 'name': 'Drama'}]"
879,300854823,70000000,230854823,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '..."
891,70600000,8500000,62100000,"[{'id': 18, 'name': 'Drama'}, {'id': 36, 'name..."
994,3251794,2200000,1051794,"[{'id': 80, 'name': 'Crime'}, {'id': 18, 'name..."
1262,53653224,30000000,23653224,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam..."
1427,132180323,50000000,82180323,"[{'id': 80, 'name': 'Crime'}, {'id': 14, 'name..."


In [133]:
print(hoffman_df.shape)
hoffman_df.head(5)

(108, 4)


,title,popularity,vote_average,vote_count
380,Rain Man,17.248,7.7,3317
586,Wag the Dog,10.386,6.9,459
693,Meet the Fockers,15.987,6.2,2795
819,Sleepers,13.808,7.5,1484
866,Finding Neverland,13.419,7.3,2352


In [134]:
print(hoffman_movies_df.shape)
hoffman_movies_df.head(5)

(108, 4)


,revenue,budget,profit,genres
380,412800000,25000000,387800000,"[{'id': 18, 'name': 'Drama'}]"
586,64256513,15000000,49256513,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam..."
693,516642939,80000000,436642939,"[{'id': 35, 'name': 'Comedy'}, {'id': 10749, '..."
819,165615285,44000000,121615285,"[{'id': 80, 'name': 'Crime'}, {'id': 18, 'name..."
866,116766556,25000000,91766556,"[{'id': 18, 'name': 'Drama'}]"


In [135]:
# This builds the full df, still need to build final
hoffman_full_df = pd.merge(hoffman_df, hoffman_movies_df, left_index=True, right_index=True)
print(hoffman_full_df.shape)
hoffman_full_df

(108, 8)


,title,popularity,vote_average,vote_count,revenue,budget,profit,genres
380,Rain Man,17.248,7.7,3317,412800000,25000000,387800000,"[{'id': 18, 'name': 'Drama'}]"
586,Wag the Dog,10.386,6.9,459,64256513,15000000,49256513,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam..."
693,Meet the Fockers,15.987,6.2,2795,516642939,80000000,436642939,"[{'id': 35, 'name': 'Comedy'}, {'id': 10749, '..."
819,Sleepers,13.808,7.5,1484,165615285,44000000,121615285,"[{'id': 80, 'name': 'Crime'}, {'id': 18, 'name..."
866,Finding Neverland,13.419,7.3,2352,116766556,25000000,91766556,"[{'id': 18, 'name': 'Drama'}]"
879,Hook,17.253,6.9,3083,300854823,70000000,230854823,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '..."
891,All the President's Men,21.243,7.7,807,70600000,8500000,62100000,"[{'id': 18, 'name': 'Drama'}, {'id': 36, 'name..."
994,Straw Dogs,12.695,7.2,406,3251794,2200000,1051794,"[{'id': 80, 'name': 'Crime'}, {'id': 18, 'name..."
1262,Stranger Than Fiction,8.599,7.2,1242,53653224,30000000,23653224,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam..."
1427,Perfume: The Story of a Murderer,12.963,7.2,2355,132180323,50000000,82180323,"[{'id': 80, 'name': 'Crime'}, {'id': 14, 'name..."


In [136]:
def to_list(cell):
    l = []
    for d in cell:     #cell is list of dicts
        l.append(d['name'])
    return(l)

hoffman_full_df['genre_names'] = hoffman_full_df['genres'].map(to_list)
hoffman_full_df

,title,popularity,vote_average,vote_count,revenue,budget,profit,genres,genre_names
380,Rain Man,17.248,7.7,3317,412800000,25000000,387800000,"[{'id': 18, 'name': 'Drama'}]",[Drama]
586,Wag the Dog,10.386,6.9,459,64256513,15000000,49256513,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...","[Comedy, Drama]"
693,Meet the Fockers,15.987,6.2,2795,516642939,80000000,436642939,"[{'id': 35, 'name': 'Comedy'}, {'id': 10749, '...","[Comedy, Romance]"
819,Sleepers,13.808,7.5,1484,165615285,44000000,121615285,"[{'id': 80, 'name': 'Crime'}, {'id': 18, 'name...","[Crime, Drama, Thriller]"
866,Finding Neverland,13.419,7.3,2352,116766556,25000000,91766556,"[{'id': 18, 'name': 'Drama'}]",[Drama]
879,Hook,17.253,6.9,3083,300854823,70000000,230854823,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...","[Adventure, Fantasy, Comedy, Family]"
891,All the President's Men,21.243,7.7,807,70600000,8500000,62100000,"[{'id': 18, 'name': 'Drama'}, {'id': 36, 'name...","[Drama, History, Mystery, Thriller]"
994,Straw Dogs,12.695,7.2,406,3251794,2200000,1051794,"[{'id': 80, 'name': 'Crime'}, {'id': 18, 'name...","[Crime, Drama, Thriller]"
1262,Stranger Than Fiction,8.599,7.2,1242,53653224,30000000,23653224,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...","[Comedy, Drama, Fantasy, Romance]"
1427,Perfume: The Story of a Murderer,12.963,7.2,2355,132180323,50000000,82180323,"[{'id': 80, 'name': 'Crime'}, {'id': 14, 'name...","[Crime, Fantasy, Drama]"


In [139]:
hoffman_full_df.drop(['genres'], axis=1, inplace=True)
hoffman_full_df

,title,popularity,vote_average,vote_count,revenue,budget,profit,genre_names
380,Rain Man,17.248,7.7,3317,412800000,25000000,387800000,[Drama]
586,Wag the Dog,10.386,6.9,459,64256513,15000000,49256513,"[Comedy, Drama]"
693,Meet the Fockers,15.987,6.2,2795,516642939,80000000,436642939,"[Comedy, Romance]"
819,Sleepers,13.808,7.5,1484,165615285,44000000,121615285,"[Crime, Drama, Thriller]"
866,Finding Neverland,13.419,7.3,2352,116766556,25000000,91766556,[Drama]
879,Hook,17.253,6.9,3083,300854823,70000000,230854823,"[Adventure, Fantasy, Comedy, Family]"
891,All the President's Men,21.243,7.7,807,70600000,8500000,62100000,"[Drama, History, Mystery, Thriller]"
994,Straw Dogs,12.695,7.2,406,3251794,2200000,1051794,"[Crime, Drama, Thriller]"
1262,Stranger Than Fiction,8.599,7.2,1242,53653224,30000000,23653224,"[Comedy, Drama, Fantasy, Romance]"
1427,Perfume: The Story of a Murderer,12.963,7.2,2355,132180323,50000000,82180323,"[Crime, Fantasy, Drama]"


In [141]:
print(hoffman_full_df.shape)
hoffman_full_df.describe()

(108, 8)


,popularity,vote_average,vote_count,revenue,budget,profit
count,108.000000,108.000000,108.000000,1.080000e+02,1.080000e+02,1.080000e+02
mean,6.241528,5.613889,530.879630,5.448879e+07,1.809667e+07,3.639213e+07
std,5.731138,2.733847,1034.010581,1.268350e+08,3.406942e+07,1.014932e+08
min,0.600000,0.000000,0.000000,0.000000e+00,0.000000e+00,-4.800000e+07
25%,0.894750,5.575000,1.000000,0.000000e+00,0.000000e+00,0.000000e+00
50%,5.508000,6.450000,63.500000,0.000000e+00,0.000000e+00,0.000000e+00
75%,10.425500,7.200000,515.000000,4.779270e+07,2.125000e+07,1.923871e+07
max,21.495000,10.000000,6402.000000,6.656923e+08,1.500000e+08,5.156923e+08


Still need to delete rows with non-movies (eg. Documentary(99), TV Movie(10770) ). Deciding what constitutes a genre.
1. Eliminate rows containing Documentery or TV Movie as a genre.
2. Eliminate rows where revenue or budget are zero.
3. Exclude Music(10402) from consideration because it doesn't say what kind of genre. It could be a dramatic, romantic or acomedy, and adventure etc... I have not seen a movie classified as solely music. So, perhaps don't delete it, just think of it as a secondary classification, at least initially.
4. Cluster analysis:
    A. Drama(18) - could include Crime(80), and War(10752);
    B. Comedy(35);
    C. Romance(10749);
    C. Family(10751) and Animation(16);
    D. Western(37);
    E. Action(28)/Adventure(12);
    F. Science Fiction(878);
    G. Horror(27) and Mystery(9648) and Thriller(53).

In [142]:
hoffman_full_df.head(8)

,title,popularity,vote_average,vote_count,revenue,budget,profit,genre_names
380,Rain Man,17.248,7.7,3317,412800000,25000000,387800000,[Drama]
586,Wag the Dog,10.386,6.9,459,64256513,15000000,49256513,"[Comedy, Drama]"
693,Meet the Fockers,15.987,6.2,2795,516642939,80000000,436642939,"[Comedy, Romance]"
819,Sleepers,13.808,7.5,1484,165615285,44000000,121615285,"[Crime, Drama, Thriller]"
866,Finding Neverland,13.419,7.3,2352,116766556,25000000,91766556,[Drama]
879,Hook,17.253,6.9,3083,300854823,70000000,230854823,"[Adventure, Fantasy, Comedy, Family]"
891,All the President's Men,21.243,7.7,807,70600000,8500000,62100000,"[Drama, History, Mystery, Thriller]"
994,Straw Dogs,12.695,7.2,406,3251794,2200000,1051794,"[Crime, Drama, Thriller]"


In [ ]:
# If Genres is in Documentary(99), TV Movie(10770), delete the row.
# If revenue, budget or profit is zero or NaN, eliminate the row.
# If vote_count	< 20, eliminate the row.


In [73]:
exclude_genre = ['TV Movie','Documentary']
delete_rows = []
list_of_genres = (row['genres'])
for genre in list_of_genres:     
    if genre['name'] in exclude_genre:
        print(genre['name'])
        print(hoffman_full_df.loc[idx])         
        delete_rows.append(idx)
print('The # of rows to exclude is: {}'.format(len(delete_rows)))
hoffman_full_df.shape

Documentary


KeyError: 371556

In [70]:
hoffman_full_df.drop(delete_rows, inplace=True)
hoffman_full_df.shape

(107, 8)

title                                           Hal
popularity                                     2.44
vote_average                                      0
vote_count                                        1
revenue                                           0
budget                                            0
profit                                            0
genres          [{'id': 99, 'name': 'Documentary'}]
Name: 322520, dtype: object
title                                           Hal
popularity                                     2.44
vote_average                                      0
vote_count                                        1
revenue                                           0
budget                                            0
profit                                            0
genres          [{'id': 99, 'name': 'Documentary'}]
Name: 322520, dtype: object
title                                           Hal
popularity                                     2.44
vote_ave

Name: 80489, dtype: object
title                         The Magnificent Rebel
popularity                                      0.6
vote_average                                    5.8
vote_count                                        4
revenue                                           0
budget                                            0
profit                                            0
genres          [{'id': 99, 'name': 'Documentary'}]
Name: 80489, dtype: object
title                         The Magnificent Rebel
popularity                                      0.6
vote_average                                    5.8
vote_count                                        4
revenue                                           0
budget                                            0
profit                                            0
genres          [{'id': 99, 'name': 'Documentary'}]
Name: 80489, dtype: object
title                         The Magnificent Rebel
popularity                         

Name: 132507, dtype: object
title                              Against the Tide
popularity                                      0.6
vote_average                                      7
vote_count                                        1
revenue                                           0
budget                                            0
profit                                            0
genres          [{'id': 99, 'name': 'Documentary'}]
Name: 132507, dtype: object
title                              Against the Tide
popularity                                      0.6
vote_average                                      7
vote_count                                        1
revenue                                           0
budget                                            0
profit                                            0
genres          [{'id': 99, 'name': 'Documentary'}]
Name: 132507, dtype: object
title                              Against the Tide
popularity                      

Name: 189203, dtype: object
title                           All the President's Men Revisited
popularity                                                  0.934
vote_average                                                  7.5
vote_count                                                      3
revenue                                                         0
budget                                                          0
profit                                                          0
genres          [{'id': 99, 'name': 'Documentary'}, {'id': 107...
Name: 189203, dtype: object
title                           All the President's Men Revisited
popularity                                                  0.934
vote_average                                                  7.5
vote_count                                                      3
revenue                                                         0
budget                                                          0
profit              

Name: 60230, dtype: object
title                                        Trumbo
popularity                                    2.596
vote_average                                    7.1
vote_count                                       10
revenue                                           0
budget                                            0
profit                                            0
genres          [{'id': 99, 'name': 'Documentary'}]
Name: 60230, dtype: object
title                                        Trumbo
popularity                                    2.596
vote_average                                    7.1
vote_count                                       10
revenue                                           0
budget                                            0
profit                                            0
genres          [{'id': 99, 'name': 'Documentary'}]
Name: 60230, dtype: object
title                                        Trumbo
popularity                         

...

Name: 527620, dtype: object
title           Controversy and Acclaim: The Timelessness of a...
popularity                                                    0.6
vote_average                                                    0
vote_count                                                      0
revenue                                                         0
budget                                                          0
profit                                                          0
genres                        [{'id': 99, 'name': 'Documentary'}]
Name: 527620, dtype: object
title           Controversy and Acclaim: The Timelessness of a...
popularity                                                    0.6
vote_average                                                    0
vote_count                                                      0
revenue                                                         0
budget                                                          0
profit              

Name: 514638, dtype: object
title                         The Making of Tootsie
popularity                                      0.6
vote_average                                      0
vote_count                                        0
revenue                                           0
budget                                            0
profit                                            0
genres          [{'id': 99, 'name': 'Documentary'}]
Name: 514638, dtype: object
title                         The Making of Tootsie
popularity                                      0.6
vote_average                                      0
vote_count                                        0
revenue                                           0
budget                                            0
profit                                            0
genres          [{'id': 99, 'name': 'Documentary'}]
Name: 514638, dtype: object
title                         The Making of Tootsie
popularity                      

Name: 588296, dtype: object
title                         Arthur Miller: A Man of His Century
popularity                                                    0.6
vote_average                                                    7
vote_count                                                      1
revenue                                                         0
budget                                                          0
profit                                                          0
genres          [{'id': 99, 'name': 'Documentary'}, {'id': 107...
Name: 588296, dtype: object
title                         Arthur Miller: A Man of His Century
popularity                                                    0.6
vote_average                                                    7
vote_count                                                      1
revenue                                                         0
budget                                                          0
profit              

Name: 617297, dtype: object
title           The Magic of Hollywood... Is the Magic of People
popularity                                                   0.6
vote_average                                                   0
vote_count                                                     0
revenue                                                        0
budget                                                         0
profit                                                         0
genres                       [{'id': 99, 'name': 'Documentary'}]
Name: 617297, dtype: object
title           The Magic of Hollywood... Is the Magic of People
popularity                                                   0.6
vote_average                                                   0
vote_count                                                     0
revenue                                                        0
budget                                                         0
profit                            

Name: 371556, dtype: object
title              Warren Beatty - Mister Hollywood
popularity                                    0.977
vote_average                                    3.5
vote_count                                        2
revenue                                           0
budget                                            0
profit                                            0
genres          [{'id': 99, 'name': 'Documentary'}]
Name: 371556, dtype: object
title              Warren Beatty - Mister Hollywood
popularity                                    0.977
vote_average                                    3.5
vote_count                                        2
revenue                                           0
budget                                            0
profit                                            0
genres          [{'id': 99, 'name': 'Documentary'}]
Name: 371556, dtype: object
title              Warren Beatty - Mister Hollywood
popularity                      